### Análisis Financiero de Plan de Negocios
En este notebook se construye un modelo que permita calcular el Flujo de Caja (Cash Flow) y otros indicadores financieros tales como: ROI (Return On Investment), IR (Indice de Rentabilidad) y Payback Period, de una ruta combinada (compuesta por VE y autos convencionales) y ver la composición de está flota para el alcance de estos indicadores.

Se importan las librerias necesarias.

In [ ]:
import pandas as pd 

Se importan los datasets que contienen la información requerida para el análisis.

In [ ]:
df_autos = pd.read_csv("df_modelo.csv") # Dataset de VE
future_data = pd.read_csv(r"Prediccion_Dataset.csv", index_col = 0) # Resultados del modelo ML (pronostico a 5 años)

Se obtiene el promedio de las variables (ingreso anual y millas recorridas por año) para un primer acercamiento y entendimiento del comportamiento de las variables.

In [4]:
promedio_ingresos = round(future_data["Income_per_Vehicle (USD)"].mean(),2)
promedio_millas = round(future_data["Miles_per_Vehicle"].mean(),2)
print(f'El promedio de ingresos anuales es: {promedio_ingresos}')
print(f'El promedio de millas recorridas anualmente es : {promedio_millas}')

El promedio de ingresos anuales es: 64271.22
El promedio de millas recorridas anualmente es : 11653.4


A continuación se asigna a cada dato una variable que almacena su información.

In [5]:
año = 2025
for _, row in future_data.iterrows():  # iterrows() en lugar de interrows()
    globals()[f'Ingresos_año_{año}'] = row["Income_per_Vehicle (USD)"]
    print(f'Los ingresos para el año {año} son {globals()[f"Ingresos_año_{año}"]}')
    año += 1

Los ingresos para el año 2025 son 65357.06
Los ingresos para el año 2026 son 64802.76
Los ingresos para el año 2027 son 64302.32
Los ingresos para el año 2028 son 63485.52
Los ingresos para el año 2029 son 63408.43


In [6]:
ingresos_años = []

for _, row in future_data.iterrows():
    ingresos_años.append(row["Income_per_Vehicle (USD)"])
print(ingresos_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43]


In [7]:
año = 2025

for _, row in future_data.iterrows():
    globals()[f'Millas_Año_{año}'] = row["Miles_per_Vehicle"]
    print(f'Las Millas recorridas en el año {año} son {globals()[f"Millas_Año_{año}"]}')
    año+=1

Las Millas recorridas en el año 2025 son 11762.56
Las Millas recorridas en el año 2026 son 11703.94
Las Millas recorridas en el año 2027 son 11656.52
Las Millas recorridas en el año 2028 son 11556.92
Las Millas recorridas en el año 2029 son 11587.08


In [8]:
millas_años = []
for _, row in future_data.iterrows():
    millas_años.append(row["Miles_per_Vehicle"])
print(millas_años)

[11762.56, 11703.94, 11656.52, 11556.92, 11587.08]


### Clases:

Se crea una clase «Auto» que contiene caracteristicas generales de este objeto. Luego en cada clase que hereda (Auto Electrico y convencional) se personalizan estas propiedades y se agregan nuevas.

Tambien se crean las funciones para el calculo del Cash Flow y los demas indicadores financieros.

In [ ]:
class Auto:
    def __init__(self, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, ingresos_anuales, tasa_descuento):
        self.precio = precio
        self.iva = iva
        self.licencia_1 = licencia_1
        self.licencia_2 = licencia_2
        self.placa = placa
        self.inspeccion = inspeccion
        self.seguro = seguro
        self.recorrido_anual = recorrido_anual
        self.ingresos_anuales = ingresos_anuales
        self.tasa_descuento = tasa_descuento
    
    def inversion_inicial(self):
        raise NotImplementedError
    
    def costos_operativos(self):
        raise NotImplementedError
    
    def flujo_neto_anual(self):
        return self.ingresos_anuales - self.costos_operativos()
    
    def flujo_caja_proyectado(self, años=5):
        # Inversión inicial negativa porqué es el año 0
        inversion_inicial = -self.inversion_inicial()
        
        
        ingresos_año = [0]
        costos_año = [0]
        flujo_neto = [0]  # Año 0: 
        flujo_descuento = [inversion_inicial]  # Año 0 Inversión inicial negativa
        # Se calcula el flujo neto anual y mensual para cada año
        for año in range(1, años + 1):
            self.ingresos_anuales = ingresos_años[año-1] # Codigo nuevo
            self.recorrido_anual = millas_años[año-1] # Codigo nuevo
            flujo_neto_actual = self.flujo_neto_anual()
            flujo_neto.append(flujo_neto_actual)
            flujo_descuento_actual = flujo_neto_actual / (1 + self.tasa_descuento) ** año
            flujo_descuento.append(flujo_descuento_actual)
            ingresos_año.append(self.ingresos_anuales)
            costos_año.append(self.costos_operativos())
        
        # Se crea el DataFrame de resultados
        df_flujo_caja = pd.DataFrame({
            'Año': range(0, años + 1),
            'Ingresos': ingresos_año,
            'Costos Operativos': costos_año,
            'Flujo Neto': flujo_neto,
            'Flujo Neto Descontado (10%)': flujo_descuento
        })
         # Calculo el flujo total descontado para el ROI
        flujo_total = sum(flujo_descuento)
        return df_flujo_caja

    def calcular_roi(self, flujo_total):
        roi = (flujo_total - self.inversion_inicial()) / self.inversion_inicial()
        return round(roi, 2)

    def calcular_ir(self, flujo_total):
        ir = flujo_total / abs(self.inversion_inicial())
        return round(ir, 2)

    def calcular_payback_period(self):
        flujo_acumulado = 0
        payback = 0
        for año in range(1, 11):
            flujo_anual = self.flujo_neto_anual()
            flujo_acumulado += flujo_anual
            if flujo_acumulado >= abs(self.inversion_inicial()):
                payback = año
                break
        return payback if payback > 0 else "No recuperado en 10 años"


Creación de clase de «Auto Convencional»

In [10]:
class AutoConvencional(Auto): #Licencia_1 = unico pago, licencia_2 = anual
    def __init__(self, precio=38000, iva=0.08875, licencia_1=550,licencia_2=100, placa = 300,
                 seguro=2500, inspeccion = 150,
                 eficiencia=0.06, mantenimiento=0.15, recorrido_anual=promedio_millas, #mantenimiento = USD x Milla
                 ingresos_anuales=promedio_ingresos, tasa_descuento=0.10):
        super().__init__(precio, iva, licencia_1, licencia_2, placa, inspeccion,
                         seguro, recorrido_anual, ingresos_anuales, tasa_descuento) # Se usa el super constructor para evitar redundancia ya que es una clase hija
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
    
    def inversion_inicial(self):
        return self.precio * (1 + self.iva) + self.licencia_1
    
    def costos_operativos(self):
        precio_gasolina_litro = 0.92 #USD x lt
        costo_gasolina_anual = (self.recorrido_anual * self.eficiencia) * precio_gasolina_litro
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        return costo_gasolina_anual + costo_mantenimiento + self.seguro + self.licencia_2 + self.placa + self.inspeccion


Creación de clase de «Auto Electrico»

In [11]:
class AutoElectrico(Auto): #Descuento de 5% al 10% en licencia ; Excencion del impuesto de venta
    def __init__(self, precio, eficiencia, iva=0, licencia_1=495,licencia_2=100, placa = 300,
                 seguro=1500, inspeccion = 100, mantenimiento=0.03, wallbox=600, 
                 recambio_bateria=5000, #mantenimiento = USD x Milla
                 recorrido_anual=16529, ingresos_anuales=93227, tasa_descuento=0.10):
        super().__init__(precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, ingresos_anuales, tasa_descuento)
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
        self.wallbox = wallbox
        self.recambio_bateria = recambio_bateria
    
    def inversion_inicial(self):
        return self.precio + self.licencia_1 + self.wallbox
    
    def costos_operativos(self):
        precio_electricidad_kWh = 0.13 #USD x kHw
        costo_electricidad_anual = (self.recorrido_anual * self.eficiencia) * precio_electricidad_kWh
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_bateria = self.recambio_bateria / 12.5
        return costo_electricidad_anual + costo_mantenimiento + costo_anual_bateria + self.seguro + self.licencia_2 + self.placa + self.inspeccion


### Modelo #3:
Análisis Financiero de Flota Combinada (VE y auto convencional)

Esta función estudia el comportamiento de cada uno de los prototipos del Dataset de VE relacionado a los indicadores financieros anteriormente delineados.
En cuanto al auto convencional se construye un propotipo prototipo como muestra.

In [ ]:
def calcular_metricas_flota(cantidad_ve, cantidad_conv):
    resultados = []

    for _, fila in df_autos.iterrows():
        # Se extraen los datos del Dataset
        modelo = fila['Model']
        precio_ve = fila['Precio Dolar']
        eficiencia_ve = fila['Efficiency (kWh/mile)']
        
        # Se instancian los autos segun su naturaleza (electrico o convencional)
        auto_electrico = AutoElectrico(precio=precio_ve, eficiencia=eficiencia_ve)
        auto_convencional = AutoConvencional()
        
        # Se calcula el flujo de caja proyectado para cada tipo de auto
        flujo_ve = auto_electrico.flujo_caja_proyectado()
        flujo_conv = auto_convencional.flujo_caja_proyectado()
        
        # Se combinar flujos de caja para la flota (teniendo en cuenta su respectiva cantidad)
        flujo_neto_comb = (
            cantidad_ve * flujo_ve['Flujo Neto Descontado (10%)'].sum() +
            cantidad_conv * flujo_conv['Flujo Neto Descontado (10%)'].sum()
        )
        
        # Se calcula  la inversión inicial combinada
        inversion_comb = (
            cantidad_ve * auto_electrico.inversion_inicial() +
            cantidad_conv * auto_convencional.inversion_inicial()
        )
        
        # Se calcular las métricas financieras combinadas
        roi_comb = (flujo_neto_comb - inversion_comb) / inversion_comb
        ir_comb = flujo_neto_comb / abs(inversion_comb)
        
        # Calculo del payback period combinado (considerando la proporción)
        flujo_anual_comb = [
            cantidad_ve * flujo_ve['Flujo Neto'].iloc[año] +
            cantidad_conv * flujo_conv['Flujo Neto'].iloc[año]
            for año in range(1, 6)
        ]
        flujo_acumulado = 0
        payback_comb = 0
        for año, flujo in enumerate(flujo_anual_comb, start=1):
            flujo_acumulado += flujo
            if flujo_acumulado >= abs(inversion_comb):
                payback_comb = año
                break
        
        # Se almacenan los resultados para el modelo actual
        resultados.append({
            'Modelo': modelo,
            'Flujo Neto Total (Millones USD/5 años)': round(flujo_neto_comb / 1e6,2),
            'ROI (%)': round(roi_comb, 2),
            'IR (USD)': round(ir_comb, 2),
            'Payback Period (años)': payback_comb if payback_comb > 0 else "No recuperado en 10 años"
        })

    # Se convertierten los resultados a tipo DataFrame para una mejor visualización
    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.sort_values(by="ROI (%)", ascending=False).reset_index(drop=True) # Se ordena para tener el ranking 


    return df_resultados


Ejemplo de uso:

In [ ]:
resultados_flota = calcular_metricas_flota(1400, 2) # Primer Parámetro : Cantidad VE; Segundo Parámetro : Cantidad Autos Convencional
resultados_flota.head(5) # head (5) para obtener el Ranking 


,Modelo,Flujo Neto Total (Millones USD/5 años),ROI (%),IR (USD),Payback Period (años)
0,Mii Electric,295.16,8.37,9.37,1
1,EQ fortwo coupe,293.28,7.79,8.79,1
2,e-Up!,293.24,7.78,8.78,1
3,EQ forfour,292.29,7.52,8.52,1
4,CITIGOe iV,288.62,6.59,7.59,1
